# 맵 추상 자료형 3부

## 균형 이진탐색트리(AVL 트리)

앞서 살펴 본 이진탐색트리의 탐색(`get()`) 및 항목추가(`put()`) 알고리즘은
최악의 경우 $O(n)$의 시간복잡도를 갖는다. 
여기서는 이진트리의 좌우 자식 서브트리를 균형이 잡히도록 항목을 추가하여
탐색과 항목추가 모두 $O(\log n)$의 시간복잡도를 갖도록 알고리즘을 구현한다.
이와같이 균형 잡힌 이진트리를 **AVL 트리**(Adelson-Velski 트리)라고 부른다.

AVL 트리를 구성하려면 모든 마디에 대해 **균형인수**(balance factor)를 
항상 평가해야 한다. 
마디의 균형인수는 해당 마디의 왼쪽 서브트리의 높이에서 
오른쪽 서브트리의 높이를 뺀 값이다.

    마디의 균형인수 = (왼쪽 서브트리의 높이) - (오른쪽 서브트리의 높이)
    
- 균형인수 $>$ 0: 왼쪽으로 치우친 트리
- 균형인수 $=$ 0: 균형 잡힌 트리
- 균형인수 $<$ 0: 오른쪽으로 치우친 트리

아래 그림은 오른쪽으로 치우친 트리에서 각 마디의 균형인수를 보여준다.

<figure>
<div align="center"><img src="https://runestone.academy/runestone/books/published/pythonds3/_images/unbalanced.png" width="35%"></div>
</figure>

### AVL 트리 분석

AVL 트리(균형 이진탐색트리)는 모든 마디의 균형인수가 -1, 0, 1 중에 하나이어야 한다.
아래 그림은 높이가 0, 1, 2, 3 인 AVL 트리를 대상으로 했을 때 왼쪽 오른쪽으로 가장 많이 
치우친 트리를 보여준다.

<figure>
<div align="center"><img src="https://runestone.academy/runestone/books/published/pythonds3/_images/worstAVL.png" width="70%"></div>
</figure>

위 그림에서 알 수 있듯이 높이가 $h$인 AVL 트리의 크기(마디 수) $N_h$는 아래 점화식을 만족한다. 

$$N_h = 1 + N_{h-1} + N_{h-2}$$

$h$가 매우 크면 아래 식이 성립한다. 
$\Phi$는 황금비(Golden Ratio)를 나타내며 피보나치 수열과 밀접하게 관련된다.
보다 자세한 설명은 [AVL Tree Performance](https://runestone.academy/runestone/books/published/pythonds3/Trees/AVLTreePerformance.html)를 참조할 수 있다.

$$
\begin{align*}
N_h  &= \frac{\Phi^{h+2}}{\sqrt{5}} - 1 \\
\Phi &= \frac{1 + \sqrt{5}}{2}
\end{align*}
$$

위 식으로부터 다음 결과가 성립하여 AVL 트리의 탐색 알고리즘이 $O(\log{N})$임을 증명한다.

$$
h  = 1.44 \log{N_h}
$$

### AVL 트리 구현

[2부](https://codingalzi.github.io/algopy/notebooks/algopy09_Map_2.html)에서 
사용한 `TreeNode`와 `BinarySearchTree`를 상속한다.

In [1]:
# 트리의 마디 클래스: 키와 값으로 이루어지며 좌우 자식과 부모 마디 정보 함께 보유
class TreeNode:
    def __init__(self, key, value, left=None, right=None, parent=None):
        self.key = key             # 키
        self.value = value         # 값
        self.left_child = left     # 좌측 자식 마디
        self.right_child = right   # 우측 자식 마디
        self.parent = parent       # 부모 마디

    # 좌측 자식 마디: 부모의 좌측 자식 마디 여부
    def is_left_child(self):
        return self.parent and self.parent.left_child is self

    # 우측 자식 마디: 부모의 우측 자식 마디 여부
    def is_right_child(self):
        return self.parent and self.parent.right_child is self

    # 루트: 부모가 없을 때
    def is_root(self):
        return not self.parent

    # 잎: 자식이 없을 때
    def is_leaf(self):
        return not (self.right_child or self.left_child)

    # 자식 존재 여부
    def has_a_child(self):
        return self.right_child or self.left_child

    # 좌우측 두 자식 모두 존재 여부
    def has_children(self):
        return self.right_child and self.left_child

    # 키, 값, 좌우 자식 지정
    def replace_value(self, key, value, left, right):
        self.key = key
        self.value = value
        self.left_child = left
        self.right_child = right
      
        # 새 자식의 부모로 자신을 지정
        if self.left_child:
            self.left_child.parent = self
        if self.right_child:
            self.right_child.parent = self

    # 현재 마디의 계승자 마디 탐색
    # 우측 자식이 있는 경우만 다뤄도 여기서는 목적 달성.
    # 모든 경우를 다루는 경우는 프로그래밍 실습 과제 참조.
    def find_successor(self):
        if self.has_children():
            successor = self.right_child.find_min()
        return successor
    
    # 현재 트리에 사용된 가장 작은 키
    def find_min(self):
        current = self
        while current.left_child:
            current = current.left_child
        return current

    # 계승자 마디 삭제 후 자식 마디들을 부모 마디에 이어 붙이기
    # 계승자는 자식을 하나만 갖기에 가능함.
    def splice_out(self):
        if self.is_leaf():
            if self.is_left_child():
                self.parent.left_child = None
            else:
                self.parent.right_child = None
        elif self.left_child:
            if self.is_left_child():
                self.parent.left_child = self.left_child
            else:
                self.parent.right_child = self.left_child
            
            self.left_child.parent = self.parent
        else:
            if self.is_left_child():
                self.parent.left_child = self.right_child
            else:
                self.parent.right_child = self.right_child
            
            self.right_child.parent = self.parent

    # 이터레이터: 중위순회방식으로 키(key) 확인
    def __iter__(self):
        if self:
            if self.left_child:
                for elem in self.left_child:
                    yield elem
            yield self.key
            if self.right_child:
                for elem in self.right_child:
                    yield elem

class BinarySearchTree:
    # 생성자: 아무 마디(TreeNode) 없는 이진탐색트리 생성. 
    # 자식 마디는 TreeNode에서 처리.
    def __init__(self):
        self.root = None    # TreeNode 객체
        self.size = 0

    # 트리 크기: 마디 개수
    def __len__(self):
        return self.size

    # 이터레이터: 모든 마디의 키를 중위순회방식으로 확인
    def __iter__(self):
        return self.root.__iter__()

    # 항목 추가: 루트 이외의 마디는 TreeNode에서 재귀적으로 처리
    def put(self, key, value):
        if not self.root:
            self.root = TreeNode(key, value)
        else:
            self._put(key, value, self.root)
        
        self.size = self.size + 1     # size를 1 키우기

    # 항목 추가 보조함수: 이진 탐색 트리 특성 유지
    def _put(self, key, value, current_node):
        if key < current_node.key:
            if current_node.left_child: 
                self._put(key, value, current_node.left_child)
            else: # 왼쪽 자식 없는 경우 마디 추가
                current_node.left_child = TreeNode(key, value, parent=current_node)
        elif key == current_node.key:  # value 업데이트
            current_node.value = value
            self.size = self.size - 1  # 키의 값을 대체하는 경우 size는 그대로 두기 위해
        else:
            if current_node.right_child:
                self._put(key, value, current_node.right_child)
            else: # 오른쪽 자식 없는 경우 마디 추가
                current_node.right_child = TreeNode(key, value, parent=current_node)

    # 사전식 키-값 지정: aBinTree[key] = value
    def __setitem__(self, key, value):
        self.put(key, value)

    #  키의 값 확인
    def get(self, key):
        if self.root:
            result = self._get(key, self.root)
            if result:
                return result.value
        return None

    # 키의 값 확인 보조함수: 키가 포함된 마디 반환. 재귀 적용
    def _get(self, key, current_node):
        if not current_node:
            return None
        if current_node.key == key:
            return current_node
        elif key < current_node.key:
            return self._get(key, current_node.left_child)
        else:
            return self._get(key, current_node.right_child)

    #  사전식 키의 값 확인: aBinTree[key]
    def __getitem__(self, key):
        return self.get(key)

    # 키 포함 여부 확인(in 연산자): key in aBinTree
    def __contains__(self, key):
        return bool(self._get(key, self.root))

    # 키 삭제: del 연산자에 활용됨
    def delete(self, key):
        # 하나의 마디만 있는 경우
        if self.size == 1 and self.root.key == key:
            self.root = None
            self.size = self.size - 1
        # 여러 마디가 있는 경우
        elif self.size > 1:
            # 삭제 대상 마디 확인
            node_to_remove = self._get(key, self.root)
            
            if node_to_remove:
                self._delete(node_to_remove)
                self.size = self.size - 1
            else:
                raise KeyError("키를 찾을 수 없음!")
        else:
            raise KeyError("키를 찾을 수 없음!")

    # 키 삭제 보조함수: 
    def _delete(self, current_node):
        # 잎 마디인 경우
        if current_node.is_leaf():
            if current_node == current_node.parent.left_child:
                current_node.parent.left_child = None
            else:
                current_node.parent.right_child = None
        
        # 좌우 자식 마디 모두 있는 경우: 계승자를 현재 마디로 옮김.
        elif current_node.has_children():
            successor = current_node.find_successor()
            successor.splice_out()
            current_node.key = successor.key
            current_node.value = successor.value
        
        # 자식 마디가 하나인 경우
        else:
            # 왼쪽 자식만 있는 경우
            if current_node.left_child:
                # 부모의 왼쪽 자식인 경우
                if current_node.is_left_child():
                    current_node.left_child.parent = current_node.parent
                    current_node.parent.left_child = current_node.left_child
                # 부모의 오른쪽 자식인 경우
                elif current_node.is_right_child():
                    current_node.left_child.parent = current_node.parent
                    current_node.parent.right_child = current_node.left_child
                # 루트인 경우
                else:
                    current_node.replace_value(
                        current_node.left_child.key,
                        current_node.left_child.value,
                        current_node.left_child.left_child,
                        current_node.left_child.right_child,
                    )
            
            # 오른쪽 자식만 있는 경우
            else:
                # 부모의 왼쪽 자식인 경우
                if current_node.is_left_child():
                    current_node.right_child.parent = current_node.parent
                    current_node.parent.left_child = current_node.right_child
                # 부모의 왼쪽 자식인 경우
                elif current_node.is_right_child():
                    current_node.right_child.parent = current_node.parent
                    current_node.parent.right_child = current_node.right_child
                # 루트인 경우
                else:
                    current_node.replace_value(
                        current_node.right_child.key,
                        current_node.right_child.value,
                        current_node.right_child.left_child,
                        current_node.right_child.right_child,
                    )
    
    # 키-값 쌍 삭제(del 연산자): del aBinTree[key]
    def __delitem__(self, key):
        self.delete(key)

**AVLTreeNode 클래스**

`TreeNode` 클래스를 상속하며,
부모 자식 관계뿐만 아니라 균형인수도 저장하도록 한다.

In [2]:
class AVLTreeNode(TreeNode):
    
    def __init__(self, key, value, balance_factor=0, left=None, right=None, parent=None):
        super().__init__(key, value, left, right, parent)
        self.balance_factor = balance_factor  # 균형인수

**AVLTree 클래스**

`BinarySearchTree` 클래스를 상속한다.

기능 업데이트 1: `_put(self, key, value, current_node)` 메서드 재정의(overriding)

새로운 항목은 항상 잎(leaf) 마디로 추가되기에 균형인수는 0이다. 
하지만 부모의 균형인수가 변하기에 적절하게 업데이트 해야 한다.

- 왼쪽 자식으로 추가되는 경우: 부모 마디의 균형인수 1 키울 것
- 오른쪽 자식으로 추가되는 경우: 부모 마디의 균형인수 1 줄일 것

위 과정을 부모의 부모 등 모든 조상에 대해 재귀적으로 적용해야 한다. 
이를 위해 `update_balance(self, node)` 메서드를 추가한다.

기능 업데이트 2: `update_balance(self, node)` 메서드 정의

새로운 자식이 추가된 부모의 균형인수를 업데이트 해야 함며,
이를 부모의 부모까지 재귀적으로 적용해야 한다. 
단, 업데이트된 부모의 균형인수가 0이라면 더 이상 재귀를 적용할 필요가 없다.
왜냐하면 업데이트된 부모의 균형인수가 0이라면 부모의 부모의 균형인수는 
전혀 바뀌지 않기 때문이다.

- 자식 추가 후 균형인수가 1보가 크거나 -1보다 작아진 경우: 더 이상 AVL 트리가 아니기에
    트리 전체를 새롭게 정돈해서 균형을 다시 잡아주어야 한다.
    이를 위해 `rebalance(self, node)` 메서드를 추가한다.

기능 업데이트 3: `rebalance(self, node)` 메서드 정의

추가된 항목으로 인해 균형이 깨진 경우 균형을 다시 잡아주도록 트리의 마디 구조를 바꾼다.
이를 위해 `rotate_left(self, rotaiton_root)`와 `rotate_right(self, rotaiton_root)`
메서드를 구현한다.

- 왼쪽으로 회전

<figure>
<div align="center"><img src="https://runestone.academy/runestone/books/published/pythonds3/_images/simpleunbalanced.png" width="50%"></div>
</figure>

- 오른쪽으로 회전

<figure>
<div align="center"><img src="https://runestone.academy/runestone/books/published/pythonds3/_images/rightrotate1.png" width="60%"></div>
</figure>

- 보다 복잡한 경우

<figure>
<div align="center"><img src="https://runestone.academy/runestone/books/published/pythonds3/_images/rotatelr.png" width="60%"></div>
</figure>

**주의**: 아래 코드는 불완전함. 

In [3]:
class AVLTree(BinarySearchTree):
    def __init__(self):
        super().__init__()

    def _put(self, key, value, current_node):
        if key < current_node.key:
            if current_node.left_child:
                self._put(key, value, current_node.left_child)
            else:
                current_node.left_child = AVLTreeNode(
                    key, value, 0, parent=current_node
                )
                self.update_balance(current_node.left_child)
        else:
            if current_node.right_child:
                self._put(key, value, current_node.right_child)
            else:
                current_node.right_child = AVLTreeNode(
                    key, value, 0, parent=current_node
                )
                self.update_balance(current_node.right_child)

    def update_balance(self, node):
        if node.balance_factor > 1 or node.balance_factor < -1:
            self.rebalance(node)
            return
        if node.parent:
            if node.is_left_child():
                node.parent.balance_factor += 1
            elif node.is_right_child():
                node.parent.balance_factor -= 1

            # 부모의 균형인수가 0이 아닌경우 재귀 적용
            if node.parent.balance_factor != 0:
                self.update_balance(node.parent)

    def rotate_left(self, rotation_root):
        new_root = rotation_root.right_child
        rotation_root.right_child = new_root.left_child
        if new_root.left_child:
            new_root.left_child.parent = rotation_root
        new_root.parent = rotation_root.parent
        if rotation_root.is_root():
            self._root = new_root
        else:
            if rotation_root.is_left_child():
                rotation_root.parent.left_child = new_root
            else:
                rotation_root.parent.right_child = new_root
        new_root.left_child = rotation_root
        rotation_root.parent = new_root
        rotation_root.balance_factor = (
            rotation_root.balance_factor + 1 - min(new_root.balance_factor, 0)
        )
        new_root.balance_factor = (
            new_root.balance_factor + 1 + max(rotation_root.balance_factor, 0)
        )

    # 수정되어야 함
    def rotate_right(self, rotation_root):
        new_root = rotation_root.left_child
        rotation_root.left_child = new_root.right_child
        if new_root.right_child:
            new_root.right_child.parent = rotation_root
        new_root.parent = rotation_root.parent
        if rotation_root.is_root():
            self._root = new_root
        else:
            if rotation_root.is_right_child():
                rotation_root.parent.right_child = new_root
            else:
                rotation_root.parent.left_child = new_root
        new_root.right_child = rotation_root
        rotation_root.parent = new_root
        rotation_root.balance_factor = (
            rotation_root.balance_factor + 1 - min(new_root.balance_factor, 0)
        )
        new_root.balance_factor = (
            new_root.balance_factor + 1 + max(rotation_root.balance_factor, 0)
        )

    def rebalance(self, node):
        if node.balance_factor < 0:
            if node.right_child.balance_factor > 0:
                self.rotate_right(node.right_child)
                self.rotate_left(node)
            else:
                self.rotate_left(node)
        elif node.balance_factor > 0:
            if node.left_child.balance_factor < 0:
                self.rotate_left(node.left_child)
                self.rotate_right(node)
            else:
                self.rotate_right(node)

## 맵 추상자료형 구현 비교

| 연산자 | 정렬 리스트 | 해시 테이블 | 이진탐색트리 | AVL 트리 |
| :---: | :---: | :---: | :---: | :---: | 
| put | O(n)      | O(1) | O(n)   | O(log n) |
| get | O(log n) | O(1) | O(n)   | O(log n) |
| in  | O(log n) | O(1) | O(n)   | O(log n) |
| del | O(n)      | O(1) | O(n)   | O(log n) |

**연습문제**

1. 아래 모양의 트리를 균형이진탐색트리로 만들어라.
    <figure>
    <div align="center"><img src="https://runestone.academy/runestone/books/published/pythonds3/_images/rotexer1.png" width="20%"></div>
    </figure>

1. 아래 이미지를 이용하여 D 마디의 균형인수를 업데이트 하는 식을 유도하라.
   
    <figure>
    <div align="center"><img src="https://runestone.academy/runestone/books/published/pythonds3/_images/bfderive.png" width="60%"></div>
    </figure>

## 프로그래밍 실습 과제

1. AVL 트리에서 항목을 삭제하는 기능을 구현하라. 
    항목삭제 후에 균형인수 업데이트와 리밸런싱 등을 적절하게 구현해야 한다.